# Visualizing CONUS404 and reference data 
 
 Author: Hannah Podzorski, USGS

 Date: 2024-04-03
 
<img src='../../../doc/assets/Eval_Viz.svg' width=600>

The purpose of visualization notebooks is to look at data in pretty ways.

<details>
  <summary>Guide to pre-requisites and learning outcomes...&lt;click to expand&gt;</summary>
  
  <table>
    <tr>
      <td>Pre-Requisites
      <td>To get the most out of this notebook, you should already have an understanding of these topics: 
        <ul>
        <li>pre-req one
        <li>pre-req two
        </ul>
    <tr>
      <td>Expected Results
      <td>At the end of this notebook, you should be able to: 
        <ul>
        <li>outcome one
        <li>outcome two
        </ul>
  </table>
</details>

In [4]:
# library imports
import os
import cf_xarray
import dask
from dask.distributed import LocalCluster, Client
import fsspec 
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
# import hvplot.pandas
import hvplot.xarray
import intake
import math
import numpy as np
import pandas as pd
import pygeohydro
import sparse 
import warnings
import xarray as xr

import panel as pn
import datetime as dt

from shapely.geometry import Polygon

warnings.filterwarnings('ignore')

# data
# connect to HyTEST catalog
url = 'https://raw.githubusercontent.com/hytest-org/hytest/main/dataset_catalog/hytest_intake_catalog.yml'
cat = intake.open_catalog(url)

# access tutorial catalog
conus404_drb_cat = cat["conus404-drb-eval-tutorial-catalog"]
list(conus404_drb_cat)

['c404-ceres-drb-desc-stats-OSN',
 'c404-crn-drb-desc-stats-OSN',
 'c404-drb-zonal-OSN',
 'c404-hcn-drb-desc-stats-OSN',
 'c404-prism-drb-desc-stats-OSN',
 'ceres-drb-OSN',
 'ceres-drb-zonal-OSN',
 'conus404-drb-OSN',
 'crn-drb-OSN',
 'crn-drb-point-OSN',
 'hcn-drb-OSN',
 'hcn-drb-point-OSN',
 'prism-drb-OSN',
 'prism-drb-zonal-OSN']

## **Start a Dask client using an appropriate Dask Cluster** 
This is an optional step, but can speed up data loading significantly, especially when accessing data from the cloud.

### Setup your client on your local PC or on HPC like this:

In [9]:
# check for existing Dask cluster
if "client" in locals():
    print("Shutting down existing Dask cluster.")
    cluster.close()
    client.close()

cluster = LocalCluster()
client = Client(cluster)

print(f"The link to the Dask dashboard is {client.dashboard_link}. If on HPC, this may not be available.")

Shutting down existing dask cluster.
The Dask dashboard link is http://127.0.0.1:8787/status


Setting up a Dask cluster for other environments will be added later. 

## Accessing already prepared CONUS404 data from OSN using `intake`

Datasets are brought into the notebook using Dask through a couple of steps. 

First, the entry (prism-drb-OSN) in the catalog (conus404_drb_cat) is indexed and the method `to_dask` will automatically load the data from the catalog entry. See below.

In [ ]:
prism_drb = conus404_drb_cat['prism-drb-OSN'].to_dask()
prism_drb

## Gridded Maps

In [9]:
date_range_slider = pn.widgets.DateRangeSlider(
    name='Date Range Slider',
    start=dt.datetime(2017, 1, 1), end=dt.datetime(2019, 1, 1),
    value=(dt.datetime(2017, 1, 1), dt.datetime(2018, 1, 10)),
    step=2
)

date_range_slider


BokehModel(combine_events=True, render_bundle={'docs_json': {'eebca90a-1d2c-45cd-ab9b-45f3b79c21ba': {'version…